## 1_でクロールしたpicleファイルをロードし、一つにまとめて、解析に使いやすいようにデータを整形したうえで再度pickleファイルで保存
- ファイルの読み込み元：'./crawledData/　以下
- ファイルの保存先：'./arrangedData/　以下

In [47]:
import glob
import pandas as pd

#### レース情報をまとめたpicleファイルを読み込んで一つにまとめる

In [48]:
# ファイル内容確認用
all_files = glob.glob('../../data/crawledData/exhibition_mod_merged/*.pkl')

# その日のレース情報をまとめたdfを要素にもつリスト
race_df_list = [pd.read_pickle(file) for file in all_files]
# concatして一つのdfにまとめる
race_df = pd.concat(race_df_list)
race_df

racer_id racer_class racer_name num_false_start  \
date       venue raceNumber 枠                                                   
2020-11-28 戸　田   1          1     3066          B1     田中　　定雄              F0   
                            2     3519          B1     冨田　　秀幸              F0   
                            3     4728          B1     山口　　　亮              F0   
                            4     4203          B1     小倉　　康典              F0   
                            5     5077          B2     滝沢　　　崚              F0   
...                                ...         ...        ...             ...   
2021-02-28 大　村   12         2     3918          A1     深井　　利寿              F0   
                            3     4330          A1     奈須　　啓太              F0   
                            4     4446          A1     和田　　兼輔              F0   
                            5     4210          A1     松村　　康太              F0   
                            6     3837          A1     秋山　　広一              F0   

                              num_late_start motorNo モーター2連率 モーター3連率 boatNo  \
date       venue raceNumber 枠                                                 
2020-11-28 戸　田   1          1             L0      53   35.44   50.63     74   
                            2             L0      48   51.02   65.31     57   
                            3             L0       2   28.21   48.72     42   
                            4             L0      14   28.17   38.03     54   
                            5             L0      45   23.88   37.31     34   
...                                      ...     ...     ...     ...    ...   
2021-02-28 大　村   12         2             L0      23   44.44   56.21     44   
                            3             L0      44   26.28   45.51     29   
                            4             L0      16   20.90   32.84     20   
                            5             L0      43   32.50   48.13     50   
                            6             L0      60   33.74   53.99     70   

                               ボート2連率  ... wave_height weight exhibitionTime  \
date       venue raceNumber 枠          ...                                     
2020-11-28 戸　田   1          1   40.51  ...           6   54.2           6.86   
                            2   37.04  ...           6   52.0           6.91   
                            3   39.44  ...           6   52.0           6.95   
                            4   32.10  ...           6   51.5           7.06   
                            5   44.05  ...           6   50.0           6.98   
...                               ...  ...         ...    ...            ...   
2021-02-28 大　村   12         2   27.21  ...           2   51.0           6.88   
                            3   36.84  ...           2   51.0           6.80   
                            4   32.12  ...           2   52.0           6.85   
                            5   25.16  ...           2   52.3           6.86   
                            6   34.18  ...           2   52.0           6.83   

                               tilt 着順     タイム 進入コース start_time  \
date       venue raceNumber 枠                                     
2020-11-28 戸　田   1          1  -0.5  ２  1'53"4   1.0       0.17   
                            2  -0.5  ４  1'55"6   2.0       0.31   
                            3  -0.5  ５           3.0       0.33   
                            4   0.0  １  1'51"4   4.0       0.21   
                            5  -0.5  ６           6.0       0.27   
...                             ... ..     ...   ...        ...   
2021-02-28 大　村   12         2  -0.5  ２  1'50"7   2.0       0.11   
                            3   0.0  １  1'47"3   3.0       0.08   
                            4   0.0  ３  1'52"4   4.0       0.07   
                            5   0.0  ５           5.0       0.14   
                            6  -0.5  ６           6.0       0.14   

                              exhib

#### 解析に使いやすいようにデータを整形する

In [55]:
from datetime import datetime as dt
import numpy as np


def arrange_exhibition_start_time(exhibition_start_time):
    # flying, lateを反映する変数を作成
    flying = False
    late = False
    if str(exhibition_start_time)[0] == "F":
        flying = True
        time = float(str(exhibition_start_time)[1:])
    elif str(exhibition_start_time)[0] == "L":
        late = True
        time = np.nan
    else:
        time = float(exhibition_start_time)
    
    return pd.Series([flying, late, time])


race_df_arranged = race_df.copy()

# それぞれの列のデータタイプの変更など
race_df_arranged["boatNo"] = race_df_arranged["boatNo"].astype("int")
race_df_arranged["motorNo"] = race_df_arranged["motorNo"].astype("int")
race_df_arranged["num_false_start"] = race_df_arranged["num_false_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["num_late_start"] = race_df_arranged["num_late_start"].map(lambda x: int(str(x)[1]))
race_df_arranged["racer_id"] = race_df_arranged["racer_id"].astype("int")
race_df_arranged["ボート2連率"] = race_df_arranged["ボート2連率"].astype("float64")
race_df_arranged["ボート3連率"] = race_df_arranged["ボート3連率"].astype("float64")
race_df_arranged["racer_class"] = race_df_arranged["racer_class"].map({"A1": 1, "A2": 2, "B1": 3, "B2": 4})
race_df_arranged["モーター2連率"] = race_df_arranged["モーター2連率"].astype("float64")
race_df_arranged["モーター3連率"] = race_df_arranged["モーター3連率"].astype("float64")
race_df_arranged["tilt"] = race_df_arranged["tilt"].astype("float64")
race_df_arranged["weight"] = race_df_arranged["weight"].astype("float64")

# exhibition start timeの列を、exhibition_fault列と、late列と、time列に分ける
race_df_arranged[["exhibition_flying", "exhibition_late", "exhibition_st"]] = race_df_arranged["exhibition_start_time"].apply(arrange_exhibition_start_time)
race_df_arranged.drop("exhibition_start_time", axis=1, inplace=True)

# weatherをdummy変数に変換
dummy_df_weather = pd.get_dummies(race_df_arranged["weather"], prefix='weather', drop_first=True)
race_df_arranged = pd.concat([race_df_arranged, dummy_df_weather], axis=1)

# 何らかの事情でデータから抜いたほうがいいレースを集める
race_df_del_list = []
# 転覆等があり、着順が6までつけられなかったレースを削除. 転覆になった選手の行だけではなくそのレースごと列を削除する
race_df_del_1 = race_df_arranged[race_df_arranged["着順"].astype(str).str.contains("\D")]
race_df_del_1.reset_index(inplace=True)
race_df_del_list.append(race_df_del_1)

# それぞれvalueに''を含むレースを削除
may_have_blank_list = [
    "exhibitionTime", "temperature", "water_temperature",
    "wave_height", "wind_speed", "exhibition_cource"
]

# 正規表現に引っ掛けるためにstr型にしておく
race_df_arranged["exhibition_cource"] = race_df_arranged["exhibition_cource"].astype("str")

for column in may_have_blank_list:
    race_df_del_ = race_df_arranged[~race_df_arranged[column].str.match("\d.*")]
    race_df_del_.reset_index(inplace=True)
    race_df_del_list.append(race_df_del_)

# Nanを含む列があるレースを削除
race_df_del_nan = race_df_arranged[race_df_arranged.isnull().any(axis=1)]
race_df_del_nan.reset_index(inplace=True)
race_df_del_list.append(race_df_del_nan)

# 消したいレースの開催地・場所・レース番号のリストを作成
remove_race_list_nested = [race_df_del[["date", "venue", "raceNumber"]].values.tolist() for race_df_del in race_df_del_list]
remove_race_list =  [item for l in remove_race_list_nested for item in l]

remove_index_list = list(set([tuple(race) for race in remove_race_list]))
race_df_arranged.drop(index=remove_index_list, inplace=True)

# 着順のカラムをinterger型に変換
race_df_arranged["着順"] = race_df_arranged["着順"].astype("int")

race_df_arranged["exhibitionTime"] = race_df_arranged["exhibitionTime"].astype("float64")
race_df_arranged["temperature"] = race_df_arranged["temperature"].astype("float64")
race_df_arranged["water_temperature"] = race_df_arranged["water_temperature"].astype("float64")
race_df_arranged["wave_height"] = race_df_arranged["wave_height"].astype("int")
race_df_arranged["wind_speed"] = race_df_arranged["wind_speed"].astype("int")
race_df_arranged["exhibition_cource"] = race_df_arranged["exhibition_cource"].astype("float64")

# multiindexは扱いがめんどくさいので全部カラムにする
race_df_arranged.reset_index(inplace=True)

# date列をdatetim型に変換
race_df_arranged["date"] = race_df_arranged["date"].map(lambda x: dt.strptime(x, '%Y-%m-%d'))

# dateとraceNumberでsort
race_df_arranged["raceNumber"] = race_df_arranged["raceNumber"].astype("float")
race_df_arranged.sort_values(["date", "venue", "raceNumber", "枠"], inplace=True)

race_df_arranged

/Users/grice/opt/anaconda3/envs/boatrace2/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


,date,venue,raceNumber,枠,racer_id,racer_class,racer_name,num_false_start,num_late_start,motorNo,...,タイム,進入コース,start_time,exhibition_cource,exhibition_flying,exhibition_late,exhibition_st,weather_曇り,weather_雨,weather_雪
5556,2020-10-03,丸 亀,2.0,1,3866,3,木山 誠一,0,0,41,...,"1'52""1",1.0,0.09,1.0,False,False,0.01,1,0,0
5557,2020-10-03,丸 亀,2.0,2,4971,4,秦 直也,0,0,57,...,"1'50""3",2.0,0.07,2.0,True,False,0.07,1,0,0
5558,2020-10-03,丸 亀,2.0,3,4552,3,杉山 勝匡,0,0,52,...,"1'56""0",3.0,0.12,3.0,False,False,0.26,1,0,0
5559,2020-10-03,丸 亀,2.0,4,3361,3,岡部 浩,0,0,60,...,"1'59""0",4.0,0.23,4.0,False,False,0.10,1,0,0
5560,2020-10-03,丸 亀,2.0,5,3617,3,竹田 広樹,0,0,63,...,"1'55""0",5.0,0.15,5.0,True,False,0.02,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30391,2021-04-04,福 岡,12.0,2,4901,2,出口 舞有子,0,0,25,...,"1'53""1",2.0,0.13,2.0,False,False,0.12,1,0,0
30392,2021-04-04,福 岡,12.0,3,4885,1,大山 千広,1,0,64,...,"1'51""7",3.0,0.11,3.0,False,False,0.11,1,0,0
30393,2021-04-04,福 岡,12.0,4,4183,2,宇野 弥生,0,0,75,...,,4.0,0.11,4.0,False,False,0.05,1,0,0
30394,2021-04-04,福 岡,12.0,5,3188,4,日高 逸子,0,0,44,...,"1'54""8",5.0,0.13,5.0,False,False,0.21,1,0,0


In [ ]:
# 統計量の算出を行うためのdf: pickleファイルで保存
race_df_arranged.to_pickle('../../data/arrangedData/data_formatted_for_statistic.pkl')

# 予測の際にそのままinputにすることができない列を削除
race_df_arranged.drop(["boatNo", "motorNo", "racer_id", "racer_name", "weather", "タイム", "start_time"], axis=1, inplace=True)

# 予測用のdf: pickleファイルで保存
race_df_arranged.to_pickle('../../data/arrangedData/data_formatted_1.pkl')